<a href="https://colab.research.google.com/github/sangmin213/DialectClassification/blob/main/Dataset_Load_Method_modified.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import random_split
import os
import re
from matplotlib import pyplot as plt
from glob import glob
import numpy as np
import pickle
from tqdm import tqdm
import time
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import KFold
from torch.utils.data import Subset

In [ ]:
index2region={0:'gangwon', 1:'gyeongsang', 2:'jeonla', 3:'chungcheong', 4:'jeju'}
region2index = {v:k for k,v in index2region.items()}
region_shortening = ['GW','GS','JL','CC','JJ']
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
# 데이터 셋 구성 (full dataset)
dataset_dir = '/content/drive/MyDrive/인공지능_프로젝트_Team_12/데이터/small_dataset/'

In [ ]:
for k, v in index2region.items():
    exec(f"{v}_dirs = glob(dataset_dir+'*_{v}/*')")
jeonla_dirs

['/content/drive/MyDrive/인공지능_프로젝트_Team_12/데이터/small_dataset/preprocessed_jeonla/DJDD20000018',
 '/content/drive/MyDrive/인공지능_프로젝트_Team_12/데이터/small_dataset/preprocessed_jeonla/DJDD20000027',
 '/content/drive/MyDrive/인공지능_프로젝트_Team_12/데이터/small_dataset/preprocessed_jeonla/DJDD20000012',
 '/content/drive/MyDrive/인공지능_프로젝트_Team_12/데이터/small_dataset/preprocessed_jeonla/DJDD20000032',
 '/content/drive/MyDrive/인공지능_프로젝트_Team_12/데이터/small_dataset/preprocessed_jeonla/DJDD20000005',
 '/content/drive/MyDrive/인공지능_프로젝트_Team_12/데이터/small_dataset/preprocessed_jeonla/DJDD20000014',
 '/content/drive/MyDrive/인공지능_프로젝트_Team_12/데이터/small_dataset/preprocessed_jeonla/DJDD20000006',
 '/content/drive/MyDrive/인공지능_프로젝트_Team_12/데이터/small_dataset/preprocessed_jeonla/DJDD20000019',
 '/content/drive/MyDrive/인공지능_프로젝트_Team_12/데이터/small_dataset/preprocessed_jeonla/DJDD20000024',
 '/content/drive/MyDrive/인공지능_프로젝트_Team_12/데이터/small_dataset/preprocessed_jeonla/DJDD20000015']

In [ ]:
# 기존의 pickle 파일을 낱개로 쪼개는 함수
def make_tuple_data(idx,dirs, max_num):
    y=[0,0,0,0,0]
    y[idx]=1
    listIDs=[] # 파일 명 저장하는 리스트
    for i, region_dir in enumerate(dirs):
        if i>=max_num:break
        spectro_path = glob(region_dir+'/*_spectro.pickle')[0]
        mfcc_path = glob(region_dir+'/*_mfcc.pickle')[0]
        chroma_path = glob(region_dir+'/*_chroma.pickle')[0]
        
        with open(spectro_path, "rb") as f:
            spectro = pickle.load(f)
        with open(mfcc_path, "rb") as f:
            mfcc = pickle.load(f)
        with open(chroma_path, "rb") as f:
            chroma = pickle.load(f)

        tuple_data=(spectro,mfcc,chroma)
        region=index2region[idx]
        
        # 낱개로 쪼개서 pickle로 다시 저장
        for index,data in enumerate(spectro):
            listIDs.append(f"{region}_{i}_{index}.pickle")
            with open(f"/content/drive/MyDrive/인공지능_프로젝트_Team_12/데이터/small_dataset/tuple_data/{region}_{i}_{index}.pickle","wb") as f:
                pickle.dump(((spectro[index],mfcc[index],chroma[index]),y),f)
        
    print(f"{region} data number:",len(listIDs))
    return listIDs

def make_tuple(max_num=2):
    listIDs=[]
    listIDs += make_tuple_data(region2index['jeonla'],jeonla_dirs, max_num)
    listIDs += make_tuple_data(region2index['chungcheong'],chungcheong_dirs, max_num)
    listIDs += make_tuple_data(region2index['gyeongsang'],gyeongsang_dirs, max_num)
    listIDs += make_tuple_data(region2index['jeju'],jeju_dirs, max_num)
    listIDs += make_tuple_data(region2index['gangwon'],gangwon_dirs, max_num)
    return listIDs

listIDs = make_tuple(1000) # listIDs에는 모든 파일 명이 저장됨

jeonla data number: 913


KeyboardInterrupt: ignored

일부러 끊었음

In [ ]:
from sklearn.preprocessing import normalize
class MultiModalDataset(Dataset):

    def __init__(self, list_IDs, path):
        self.list_IDs=list_IDs # ID = file name
        self.path=path # data directory

    def __getitem__(self, idx):
        file_name = self.list_IDs[idx] # 인덱스에 맞는 파일명 확인
        with open(path+file_name,"rb") as f: # 해당 파일 불러오기
            file=pickle.load(f)
        datas, label = file
        spec, mfcc, chroma = datas
        spec, mfcc, chroma = normalize(spec), normalize(mfcc), normalize(chroma)
        spec, mfcc, chroma = torch.tensor(spec, dtype=torch.float32), torch.tensor(mfcc, dtype=torch.float32), torch.tensor(chroma, dtype=torch.float32)
        spec, mfcc, chroma = spec.unsqueeze(0), mfcc.unsqueeze(0), chroma.unsqueeze(0)
        label = torch.tensor(label, dtype=torch.float32)

        data = (spec, mfcc, chroma)
        return data, label
    
    def __len__(self):
        return len(self.list_IDs)

In [ ]:
path="/content/drive/MyDrive/인공지능_프로젝트_Team_12/데이터/small_dataset/tuple_data/"
dataset=MultiModalDataset(listIDs,path) # 데이터셋 선언
len(dataset)

913

In [ ]:
class Block(nn.Module):
    def __init__(self,input_channel, output_channel, kernel_size, stride=1, padding=0):
        super(Block,self).__init__()

        self.conv1=nn.Conv2d(input_channel,output_channel,kernel_size=kernel_size,padding=padding) # no stride
        self.conv2=nn.Conv2d(output_channel,output_channel,kernel_size=kernel_size,stride=stride,padding=padding) # stride if down sampling
        self.bn=nn.BatchNorm2d(output_channel)
        self.relu=nn.ReLU()

        self.layer=nn.Sequential(self.conv1, self.bn, self.relu, self.conv2, self.bn)

        self.stride=stride
        self.iden = nn.Conv2d(input_channel, output_channel, kernel_size=(1,1), stride=1)
        if input_channel==64:
            self.iden = nn.Conv2d(input_channel, output_channel, kernel_size=(1,1), stride=1)

    def forward(self,x):
        y = self.layer(x)
        if self.stride==1: # stride==2 인 경우는 downsampling 구간이기 때문에 residual 안함
            y = y + self.iden(x)
        y = self.relu(y)

        return y

In [ ]:
class LeNet(nn.Module):
    def __init__(self,data_type="mfcc",method="origin",best_model_save_path="./LeNet_best_model.pt"):
        super(LeNet,self).__init__()
        
        self.best_model_save_path = best_model_save_path
        self.data_type=data_type
        self.method=method

        if data_type=="mfcc": # (1,100,501)
            self.conv1=nn.Conv2d(1,64,kernel_size=(6,7),stride=2,padding=3)
        elif data_type=="spec": #(1,201,501)
            self.conv1=nn.Conv2d(1,64,kernel_size=(7,7),stride=2,padding=3)
        elif data_type=="chroma": #(1,12,501)
            self.conv1=nn.Conv2d(1,64,kernel_size=(6,7),stride=1,padding=3)
        self.maxpool=nn.MaxPool2d(kernel_size=(3,3),stride=2,padding=1)
        self.avgpool=nn.AdaptiveAvgPool2d((1,1)) # global avg pool
        self.relu=nn.ReLU()

        self.seq1=nn.Sequential(self.conv1,self.relu,self.maxpool)  # (1,100,501)/(1,12,501)/(1,201,501) -> (64,51,251)/(1,13,251)/(1,101,251).conv -> (64,25,126)/(64,7,126)/(64,51,126).maxpool
        self.seq2=nn.Sequential(Block(64,64,(3,3),padding=1),Block(64,64,(3,3),padding=1),Block(64,128,(3,4),stride=2,padding=1)) # (64,25,126) -> (64,25,126).block -> (64,25,126).block -> (128,13,63)/(128,4,63)/(128,26,63).block 
        if data_type=="mfcc":
            self.seq3=nn.Sequential(Block(128,128,(3,3),padding=1),Block(128,128,(3,3),padding=1),Block(128,256,(3,3),stride=2,padding=1)) # (128,13,63) -> (128,13,63).block -> (128,13,63).block -> (256,7,32).block 
        elif data_type=="spec":
            self.seq3=nn.Sequential(Block(128,128,(3,3),padding=1),Block(128,128,(3,3),padding=1),Block(128,256,(4,3),stride=2,padding=1)) # (128,26,63) -> ... -> (256,12,32).block 
        elif data_type=="chroma":
            self.seq3=nn.Sequential(Block(128,128,(3,3),padding=1),Block(128,128,(3,3),padding=1),Block(128,256,(2,3),stride=2,padding=1)) # (128,4,63)/(128,26,63) -> ... -> (256,2,32)/(256,13,32).block 
        self.seq4=nn.Sequential(Block(256,256,(3,3),padding=1),Block(256,256,(3,3),padding=1),self.avgpool) # ... -> (256,1,1).avgpool

        if self.method=="multimodal":
            self.fc=nn.Linear(256,256)
        if self.method=="origin":
            self.fc=nn.Linear(256,5)
            self.loss=nn.CrossEntropyLoss()
            self.optimizer=optim.Adam(self.parameters(),lr=0.0001)

        self.train_accuracy = []
        self.train_loss = []
        self.val_accuracy = []
        self.val_loss = []

    def forward(self,x):       
        y=self.seq1(x)
        y=self.seq2(y)
        y=self.seq3(y)
        y=self.seq4(y)
        y=y.view(y.shape[0],-1)
        y=self.fc(y)
        return y

    def train_(self, train_loader, val_loader, learning_rate, epochs, device):
        self.train_accuracy = []
        self.train_loss = []
        self.val_accuracy = []
        self.val_loss = []
        self.pred_labels_train = []
        self.real_labels_train = []
        self.pred_labels_val = None
        self.real_labels_val = None

        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)
        self.loss_f=nn.CrossEntropyLoss()

        best_epoch = -1
        best_acc = -1 
        
        for epoch in range(1, epochs+1):
            total = 0
            correct = 0
            start_time = time.time()
            epoch_loss = 0.0
            epoch_acc = 0.0
            self.train()

            for batch_idx, (batch_data, batch_label) in enumerate(tqdm(train_loader)):
                
                spec, mfcc, chroma = batch_data

                if self.data_type=="mfcc":
                    batch_data=mfcc.to(device)
                elif self.data_type=="spec":
                    batch_data=spec.to(device)
                elif self.data_type=="chroma":
                    batch_data=chroma.to(device)
                
                batch_label = batch_label.to(device)

                self.optimizer.zero_grad()

                pred = self.forward(batch_data) # (batch_size, 5)
                loss = self.loss_f(pred, batch_label)
                loss.backward()
                self.optimizer.step()

                epoch_loss += loss.item()

                _, pred_indices = torch.max(pred, axis=1)
                total += batch_data.shape[0]
                batch_label = torch.max(batch_label, axis=1)[1]
                correct += pred_indices.eq(batch_label).sum().item()
                
                if epoch==epochs: #last epoch
                    self.pred_labels_train.append(pred_indices)
                    self.real_labels_train.append(batch_label)
                #for p, l in zip(pred_indices, batch_label):
                #    print(f"predicted: {index2region[p.item()]} real:{index2region[l.item()]}")
            
            end_time = time.time()
            print(f"epoch {epoch} time: {end_time-start_time}sec(s).")
            

            epoch_loss /= len(train_loader)
            self.train_loss.append(epoch_loss)
            epoch_acc = correct / total
            self.train_accuracy.append(epoch_acc)
            print(f"epoch {epoch} train accuracy: {epoch_acc}")
            print(f"epoch {epoch} loss: {epoch_loss}")  


            predicted, labels, val_loss = self.predict(val_loader, device)
            if epoch==epochs: #last epoch
                self.pred_labels_val=predicted.cpu().numpy()
                self.real_labels_val=labels.cpu().numpy()
            val_acc = predicted.eq(labels).sum().item() / len(predicted)
            print(f"epoch {epoch} val accuracy: {val_acc}")
            print(f"epoch {epoch} val loss: {val_loss}")

            if val_acc > epoch_acc:
                best_acc = val_acc
                best_epoch = epoch
                torch.save(self.state_dict(), self.best_model_save_path)
            
            self.val_accuracy.append(val_acc)
            self.val_loss.append(val_loss)
        
        self.pred_labels_train = torch.cat(self.pred_labels_train, dim=0)
        self.real_labels_train = torch.cat(self.real_labels_train, dim=0)
        self.pred_labels_train = self.pred_labels_train.cpu().numpy()
        self.real_labels_train = self.real_labels_train.cpu().numpy()
            
            
            
        print("Finish!")
        
        return best_acc, best_epoch
            
    def predict(self, test_loader, device):
        self.eval()
        labels = []
        predicted = []
        val_loss = 0.0
        with torch.no_grad():
            for batch_idx, (batch_data, batch_label) in enumerate(tqdm(test_loader)):

                spec, mfcc, chroma = batch_data
                if self.data_type=="mfcc":
                    batch_data=mfcc.to(device)
                elif self.data_type=="spec":
                    batch_data=spec.to(device)
                elif self.data_type=="chroma":
                    batch_data=chroma.to(device)
                batch_label = batch_label.to(device)
                
                pred = self.forward(batch_data)

                _, pred_indices = torch.max(pred, axis=1)
                loss = self.loss_f(pred, batch_label)
                
                val_loss += loss.item()

                predicted.append(pred_indices)
                batch_label = torch.max(batch_label, axis=1)[1]
                labels.append(batch_label)
        val_loss /= len(test_loader)
        predicted = torch.cat(predicted, dim=0)
        labels = torch.cat(labels, dim=0)

        return predicted, labels, val_loss
    
    def plot(self, which):
        
        X = [i for i in range(1, len(self.train_accuracy) + 1)]
        if which=='train_loss':
            y = self.train_loss
        elif which=='train_acc':
            y = self.train_accuracy
        elif which=='val_acc':
            y = self.val_accuracy
        elif which=='val_loss':
            y = self.val_loss
        elif which=='confusion_train':
            ConfusionMatrixDisplay.from_predictions(self.real_labels_train, self.pred_labels_train, display_labels=region_shortening)
            plt.title('train confusion matrix')
            plt.savefig(f"./result/LeNet_{which}_{self.data_type}.png")
            plt.show()
            return
        elif which=='confusion_normalize_train':
            ConfusionMatrixDisplay.from_predictions(self.real_labels_train, self.pred_labels_train, display_labels=region_shortening, normalize='true')
            plt.title('train confusion matrix')
            plt.savefig(f"./result/LeNet_{which}_{self.data_type}.png")
            plt.show()    
            return        
        elif which=='confusion_val':
            ConfusionMatrixDisplay.from_predictions(self.real_labels_val, self.pred_labels_val, display_labels=region_shortening)
            plt.title('val confusion matrix')
            plt.savefig(f"./result/LeNet_{which}_{self.data_type}.png")
            plt.show()
            return
        elif which=='confusion_normalize_val':
            ConfusionMatrixDisplay.from_predictions(self.real_labels_val, self.pred_labels_val, display_labels=region_shortening, normalize='true')
            plt.title('val confusion matrix')
            plt.savefig(f"./result/LeNet_{which}_{self.data_type}.png")
            plt.show()    
            return   
            

        plt.xlabel("epoch")
        plt.ylabel(which)
        plt.title(which)
        plt.plot(X, y, label="Train loss")
        plt.savefig(f"./result/LeNet_{which}_{self.data_type}.png")
        plt.show()

In [ ]:
def CV_Plot(title, arg, y):
        X = [i for i in range(1, len(y) + 1)]
        plt.xlabel("epoch")
        plt.ylabel(title)
        plt.title(title)
        plt.plot(X, y, label=title)
        plt.savefig(f"./result/model_{title}_{arg}.png")
        plt.show()

In [ ]:
def CrossValidation(dataset, learning_rate, epochs, device, method = "ResNet",data_type="mfcc"):
        hparams = []
        for i in range(len(learning_rate)):
            for j in range(len(epochs)):
                hparams.append((learning_rate[i], epochs[j]))
        print(hparams)

        train_dataset_l = []
        validation_dataset_l = []

        kf = KFold(n_splits = 5, shuffle = True, random_state = 50)

        for train_index, test_index in kf.split(train_dataset):
            train_dataset_l.append(Subset(train_dataset,train_index))
            validation_dataset_l.append(Subset(train_dataset,test_index))

        result = []
        for i in range(len(hparams)):
            lr = hparams[i][0]
            e = hparams[i][1]

            print(f"Learning rate : {lr}, Epochs : {e}")

            last_val_acc = []
            for j in range(5):
                print(f"#{j+1} validation")
                if method=="Multimodal_ResNet":
                    model = MultiModalDialectClassifier(method="ResNet").to(device) # 매번 새로 정의해서 다시 학습해야함
                elif method=="Multimodal_LeNet":
                    model = MultiModalDialectClassifier(method="LeNet").to(device) # 매번 새로 정의해서 다시 학습해야함
                elif method=="ResNet":
                    model = ResNet18(1,output_dim=5,model_type=data_type).to(device) # 매번 새로 정의해서 다시 학습해야함
                elif method=="LeNet":
                    model = LeNet(data_type=data_type).to(device) # 매번 새로 정의해서 다시 학습해야함
                elif method=="LSTM":
                    model = LSTM(data_type=data_type).to(device) # 매번 새로 정의해서 다시 학습해야함
                train_loader = DataLoader(train_dataset_l[j], batch_size=32, shuffle=True)
                validation_loader = DataLoader(validation_dataset_l[j], batch_size=32, shuffle=True)

                model.train_(train_loader, validation_loader, lr, e, device)
                last_val_acc.append(model.val_accuracy[-1])
                    
                # model.plot('train_acc')
                # model.plot('val_acc')
            result.append((np.array(last_val_acc)).mean())

        idx = result.index(max(result))
        best_lr, best_ep = hparams[idx]
        print(f"Best Learning Rate : {best_lr}, Best Epoch : {best_ep}")

        return best_lr, best_ep

In [ ]:
train_size = int(len(dataset)*0.8)
test_size = len(dataset) - train_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

In [ ]:
lr = [0.0001, 0.0005, 0.001]
ep = [30]

result = CrossValidation(train_dataset, learning_rate= lr, epochs= ep, device = device, method = "LeNet",data_type="mfcc")

[(0.0001, 30), (0.0005, 30), (0.001, 30)]
Learning rate : 0.0001, Epochs : 30
#1 validation


100%|██████████| 19/19 [03:19<00:00, 10.48s/it]


epoch 1 time: 199.16643905639648sec(s).
epoch 1 train accuracy: 0.958904109589041
epoch 1 loss: 0.5163380017406062


100%|██████████| 5/5 [00:14<00:00,  2.91s/it]


epoch 1 val accuracy: 1.0
epoch 1 val loss: 0.4467105448246002


  0%|          | 0/19 [00:03<?, ?it/s]


KeyboardInterrupt: ignored

일부러 끊었음